<a href="https://colab.research.google.com/github/DaAnMaGi/PI_ML_OPS/blob/main/formulacion/formulacion_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descarga de bibliotecas a utilizar

In [1]:
# Se descargan las bibliotecas necesarias
!pip install nltk
!pip install unidecode
!pip install langdetect
!pip install googletrans==4.0.0-rc1
!pip install inflect
!pip install regex

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [2]:
# Se importan las bibliotecas a utilizar

# Pandas y numpy
import pandas as pd
import numpy as np

# Trabajo de archivos
import gzip
import json
from pandas import json_normalize

# Detección de idioma
from langdetect import detect, LangDetectException
from googletrans import Translator

#
import itertools

# Gráficación
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Análisis de lenguaje natural
from unidecode import unidecode
import regex
import unicodedata
import inflect
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from nltk.corpus import wordnet


# Machine Learning
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  archivo_json = DataFrame.to_json()
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

# Se cargan las bases

In [7]:
# Se traen las rutas de los archivos a trabajar.
#games_ruta = "/content/games_clean.json.gz"
#genres_ruta = "/content/genres_games.json.gz"
#specs_ruta = "/content/specs_games.json.gz"
reviews_ruta = "/content/reviews_limpia.json.gz"
#items_ruta = "/content/items_limpia.json.gz"

In [8]:
# Se convierten a pandas.

#games = pd.read_json(games_ruta,compression="gzip")
#genres = pd.read_json(genres_ruta,compression="gzip")
#specs = pd.read_json(specs_ruta,compression="gzip")
reviews = pd.read_json(reviews_ruta,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
#items = pd.read_json(items_ruta,compression="gzip")

# Procesamiento del lenguaje natural en las reviews:

1.   Identificación de idioma de cada review.
2.   Separación de las reviews en inglés.
3.   Procesamiento de las reviews para obtenerlas lematizadas.



In [9]:
reviews

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14


In [10]:
# Se obtiene información sobre la distribución del dataset.
reviews["recommend"].value_counts(normalize=True)

True     0.906972
False    0.093028
Name: recommend, dtype: float64

In [11]:
#Se crea una función para limpiar la base de datos, buscando mantener la escritura
# de los idiomas no alfabéticos

def clean_text(text):
    # Reemplazo de caracteres comunes
    text = re.sub(r"10/10","amazing game", text)
    text = re.sub(r"<3","loved the game", text)
    text = re.sub(r":\)","good game", text)
    # Eliminar caracteres numéricos y de puntuación
    text = regex.sub(r"[^\p{L}p{Z}]", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    # Dejar el texto en minusculas
    text = text.lower()
    # Normalizar el texto a la forma NFKC (compatibilidad y composición)
    text = unicodedata.normalize('NFKC', text)

    return text

# Se crea una segunda iteración de limpieza de texto para casos que no pudieron ser limpiados.
def clean_text2(text):
  text = re.sub("[^a-zA-Z]"," ",text)
  text = re.sub(r"\s{2,}", "", text)
  return text

In [12]:
# Se limpia la base de datos.
reviews["review_clean"] = reviews["review"].apply(clean_text).replace([""," "],"no review")

In [13]:
# Se crea una función para detectar el lenguaje.
# En caso de que no sea capaz de detectarlo, no trae ningún valor.
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException as e:
        return None

In [15]:
# Se detecta el lenguaje para cada review
# reviews["language"] = reviews["review_clean"].apply(detect)
reviews["language"] = reviews["review_clean"].apply(detect_language)

In [16]:
# Para los casos de lenguaje nulo se vuelve a aplicar una segunda capa de limpieza.
reviews.loc[reviews["language"].isna(), "review_clean"] = reviews.loc[reviews["language"].isna(), "review"].apply(clean_text2).replace(["", " "], "no review")

In [17]:
# Se reemplaza el idioma de los nulos con "inglés"
reviews["language"].fillna("en",inplace=True)

In [18]:
# Se obtiene la información de los lenguajes detectados en el dataframe.
reviews["language"].unique()

array(['en', 'tl', 'so', 'sv', 'af', 'cy', 'th', 'pl', 'pt', 'sl', 'fi',
       'ru', 'da', 'id', 'lt', 'it', 'no', 'sw', 'zh-cn', 'ko', 'nl',
       'fr', 'ro', 'es', 'hr', 'cs', 'hu', 'ca', 'de', 'et', 'tr', 'sk',
       'sq', 'lv', 'bg', 'vi', 'ja', 'uk', 'zh-tw', 'mk', 'ar'],
      dtype=object)

In [19]:
# Se revisa la base obtenida.
reviews

,user_id,item_id,recommend,review,review_date,review_clean,language
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,simple yet with great replayability in my opin...,en
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,it s unique and worth a playthrough,en
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,great atmosphere the gunplay can be a bit chun...,en
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24,i know what you think when you see this title ...,en
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08,for a simple it s actually not all that simple...,en
...,...,...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14,its funnnnnnnn,no
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10,awesome fantasy game if you don t mind the gra...,en
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31,prettyy mad game,no
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14,amazing game amazing game,tl


In [20]:
reviews["language"].unique()

# Es importante notar que hay reviews (por ejemplo, del inglés y del español)
# que fueron detectadas en otros idiomas, esto debido a errores de escritura
# u ortografía.

array(['en', 'tl', 'so', 'sv', 'af', 'cy', 'th', 'pl', 'pt', 'sl', 'fi',
       'ru', 'da', 'id', 'lt', 'it', 'no', 'sw', 'zh-cn', 'ko', 'nl',
       'fr', 'ro', 'es', 'hr', 'cs', 'hu', 'ca', 'de', 'et', 'tr', 'sk',
       'sq', 'lv', 'bg', 'vi', 'ja', 'uk', 'zh-tw', 'mk', 'ar'],
      dtype=object)

In [21]:
# Tras una revisión detalla, se detecta que la mayoría de las reviews de los siguientes
# idiomas estaban en inglés
lang_change = ["tl","so","sv","af","cy","sl","fi","id","lt","no","da",
               "sw","nl","ro","hr","hu","ca","et","lv","sq"]

# Idiomas que presentan una mezcla entre reviews en inglés y de otro idioma.
lang_mix = ["pl","it","de","tr","sk"]

# Se procede a realizar una revisión para los idiomas anteriormente detectados:
for row in range(reviews.shape[0]):
  # Se cambia el idioma de los lenguajes que se detectaron mayormente en inglés.
  if reviews.loc[row,"language"] in lang_change:
    reviews.loc[row,"language"] = "en"
  # Se intenta volver a revisar el lenguaje de los datos mezclados.
  elif reviews.loc[row,"language"] in lang_mix:
    reviews.loc[row,"language"] = detect(reviews.loc[row,"review_clean"])


In [22]:
# Se decide guardar la base con los idiomas detectados.
guardar_archivo(reviews,"/content/reviews_lang.json.gz")

# Se decide seguir trabajando con la nueva base de reviews a partir del nuevo archivo
ruta_reviews_lang = "/content/reviews_lang.json.gz"
reviews = pd.read_json(ruta_reviews_lang,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")

In [24]:
# Se crea una nueva columna en reviews para guardar las traducciones.
reviews["translation"] = ""

#Se procede a traducir cada uno de las reviews.
for row in range(reviews.shape[0]):
    # Si el texto está en inglés, no se traduce.
    if reviews.loc[row, "language"] == "en":
        reviews.loc[row, "translation"] = reviews.loc[row, "review_clean"]
    # Si el texto está en otro idioma, se busca traducir al inglés.
    else:
        try:
            translator = Translator()
            translation = translator.translate(reviews.loc[row, "review_clean"], dest="en")
            reviews.loc[row, "translation"] = translation.text
        except Exception as e:
            reviews.loc[row, "translation"] = "no review"

In [26]:
# Se visualizan las traducciones de alguno de los idiomas.
reviews[reviews["language"] == "zh-cn"]

,user_id,item_id,recommend,review,review_date,review_clean,language,translation
223,2768820078,287700,True,一部小岛秀夫的游戏，游戏中居然还出现了这句话，算是konami对玩家的妥协么,2015-09-01,一部小岛秀夫的游戏 游戏中居然还出现了这句话 算是konami对玩家的妥协么,zh-cn,This sentence appeared in a game of Showfu Koh...
224,2768820078,377160,True,我想说，退款的都是sb,2015-11-18,我想说 退款的都是sb,zh-cn,I want to say that the refund is SB
225,2768820078,221380,True,"除了卖的太贵,士兵模型大多数国家都一样外，没有什么缺点",2015-08-08,除了卖的太贵 士兵模型大多数国家都一样外 没有什么缺点,zh-cn,There are no disadvantages except for being so...
227,2768820078,222420,False,如果拿来和ps2版的98um来比的话，阉割太严重，本来的3d背景居然又变回老98的2d背景了...,2015-06-12,如果拿来和ps 版的 um来比的话 阉割太严重 本来的 d背景居然又变回老 的 d背景了 语...,zh-cn,If it is comparable to the UM of the PS versio...
228,2768820078,271590,True,这代感觉恶搞的内容多了很多，外星人，僵尸，各种恶搞政府，种族歧视，资本家，环保，媒体的节目。...,2015-05-19,这代感觉恶搞的内容多了很多 外星人 僵尸 各种恶搞政府 种族歧视 资本家 环保 媒体的节目 ...,zh-cn,There are more content of spoofing this genera...
...,...,...,...,...,...,...,...,...
41865,76561198093386811,440,True,Steam上面貌似这个事最赚钱的游戏。但是这个是很好玩。,2014-06-17,steam上面貌似这个事最赚钱的游戏 但是这个是很好玩,zh-cn,no review
41868,76561198093386811,234310,False,Mac一次都没有成功打开过游戏。。。,2014-06-17,mac一次都没有成功打开过游戏,zh-cn,no review
42775,PulpCraft,369580,True,多么一个厚颜无耻的游戏……,2015-07-04,多么一个厚颜无耻的游戏,zh-cn,no review
43348,76561198122822618,271590,True,不得不说这款游戏绝对是一款吸引人的作品，虽然他两年前就在ps3和xb上有了，但丝毫也不影响我...,2015-04-23,不得不说这款游戏绝对是一款吸引人的作品 虽然他两年前就在ps 和xb上有了 但丝毫也不影响我...,zh-cn,no review


In [30]:
# Tras la traducción se decide eliminar las columnas de "review" anteriores,
# dejando únicamente la traducción y la del lenguaje.
reviews_trans = reviews.drop(columns=["review", "review_clean", "language"]).rename(columns={"translation": "review"})

In [32]:
reviews_trans

,user_id,item_id,recommend,review_date,review
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...
...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...
43973,72947282842,730,True,2015-10-31,prettyy mad game
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game


In [67]:
# Se guarda la nueva data en un nuevo documento:
# guardar_archivo(reviews_trans,"/content/reviews_en.json.gz")

# Se decide seguir trabajando con la nueva base de reviews a partir del nuevo archivo
ruta_reviews_en = "/content/reviews_en.json.gz"
reviews_en = pd.read_json(ruta_reviews_en,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")

In [68]:
reviews_en

,user_id,item_id,recommend,review_date,review
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...
...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...
43973,72947282842,730,True,2015-10-31,prettyy mad game
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game


In [38]:
# Se obtiene información sobre la distribución del dataset.
reviews_en["recommend"].value_counts(normalize=True)

True     0.906972
False    0.093028
Name: recommend, dtype: float64

In [69]:
# Se crea la lista de stopwords, como estamos trabajando con los reviews en inglés,
# trabajaremos con estos.
stopwords = nltk.corpus.stopwords.words('english')
stopwords = list(stopwords)

In [40]:
# Se agregan a la lista de stopwords, después de una primera iteración palabras
# que después no representan información útil.
filtrar = []

if True:
  filtrar.append("game")
  filtrar.append("games")

In [41]:
# se crea la siguiente función para obtener la categoría de la palabra que se usará
# para la lematización
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [88]:
# Se limpia, tokeniza y lematiza cada una de las
# reviews de la base de datos.

reviews_lemma = []

for review in reviews_en.review:
  # Se limpia de caracteres no alfabéticos
  review = re.sub("[^a-zA-Z]"," ",str(review))
  # Se limpiza de caracteres repetidos numerosas veces.
  review = re.sub(r'(.)\1{2,}', r'\1\1', review)
  # Se tokeniza.
  review = nltk.word_tokenize(review)

  # Se lematizan las palabras
  review = [wnl.lemmatize(w, get_wordnet_pos(w)) for w in review]

  # Se eliminan palabras menores de 3 letras
  review = [word for word in review if len(word)>3]
  # Sacamos las Stopwords
  review = [word for word in review if word not in stopwords]
  review = [word for word in review if word not in filtrar]

  # Se unifica nuevamente la review
  review = " ".join(review)

  # Se agrega a la lista que luego servirá para agregar la info al dataframe
  reviews_lemma.append(review)


In [89]:
# Se visualiza el proceso.
reviews_lemma[0:40]

['simple great replayability opinion zombie horde team work well left dead plus global leveling system alot earth zombie splatter whole family amaze sort rare',
 'unique worth playthrough',
 'great atmosphere gunplay chunky time definitely worth hope sequel sequel',
 'know think title barbie dreamhouse party intimidate title easily gotys cliche mechanic late simply good core gameplay noscope friend show dance move shame show true fashion color combination know easily range blast play',
 'simple actually simple truck drive simulator quite relax play simple easy basic wasd drive want much harder realistic manually change gear much harder turn reverse imagine would actual truck luckily reverse park extra point cause bloody hard suprisingly nice truck drive',
 'little play bore time passer recommend',
 'elegant integration gameplay story world development aesthetic',
 'random drop random quest stat point animation style reminiscent voodoo card',
 'balance tactic strategy potential reward b

In [90]:
# Se agrega la lematización a el dataset.
reviews_en["review_lemma"] = reviews_lemma

In [91]:
# Se visualiza el dataset
reviews_en

,user_id,item_id,recommend,review_date,review,review_lemma
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...,simple great replayability opinion zombie hord...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough,unique worth playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...,great atmosphere gunplay chunky time definitel...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...,know think title barbie dreamhouse party intim...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...,simple actually simple truck drive simulator q...
...,...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn,funn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...,awesome fantasy mind graphic recommend
43973,72947282842,730,True,2015-10-31,prettyy mad game,prettyy
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game,amaze amaze


In [92]:
# Se procede a guardar la base con los datos lematizados para futuras iteraciones:
guardar_archivo(reviews_en,"/content/reviews_lemma.json.gz")

# Análisis de las reviews lematizadas.

In [93]:
ruta_reviews_lemma = "/content/reviews_lemma.json.gz"
reviews = pd.read_json(ruta_reviews_lemma,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews

,user_id,item_id,recommend,review_date,review,review_lemma
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...,simple great replayability opinion zombie hord...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough,unique worth playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...,great atmosphere gunplay chunky time definitel...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...,know think title barbie dreamhouse party intim...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...,simple actually simple truck drive simulator q...
...,...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn,funn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...,awesome fantasy mind graphic recommend
43973,72947282842,730,True,2015-10-31,prettyy mad game,prettyy
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game,amaze amaze


In [109]:
# Se extraen las lematizaciones
review_lemma = reviews[["recommend","review_lemma"]]
review_lemma.head()

,recommend,review_lemma
0,True,simple great replayability opinion zombie hord...
1,True,unique worth playthrough
2,True,great atmosphere gunplay chunky time definitel...
3,True,know think title barbie dreamhouse party intim...
4,True,simple actually simple truck drive simulator q...


In [110]:
# Creamos el vector que nos ayudará a vectorizar las reviews.
vec = CountVectorizer(max_features=1000,stop_words="english",ngram_range=(1,1))
# En este caso, el vector trae únicamente las 1000 palabras con más frecuencia.
# Dejando ngramas de 1 palabra.
vec

CountVectorizer(max_features=1000, stop_words='english')

In [111]:
# Se alimenta el vector con la base de datos.
modelo_reviews = vec.fit_transform(review_lemma["review_lemma"])

In [112]:
# Se obtienen las palabras aprendidas por el modelo.
words = vec.get_feature_names_out()
# Se obtiene la matriz
matrix = modelo_reviews.toarray()

In [113]:
words[0:100]

array(['ability', 'able', 'absolute', 'absolutely', 'access', 'account',
       'achievement', 'action', 'actual', 'actually', 'addict',
       'addictive', 'addition', 'addons', 'advantage', 'adventure',
       'ahead', 'alien', 'alive', 'allow', 'allows', 'alot', 'alpha',
       'alright', 'amaze', 'amazingly', 'ammo', 'animal', 'animation',
       'annoy', 'answer', 'anymore', 'apart', 'appear', 'arcade', 'area',
       'arkham', 'arma', 'armor', 'armour', 'army', 'aside', 'aspect',
       'assassin', 'aswell', 'atmosphere', 'attack', 'attempt',
       'attention', 'australia', 'australian', 'available', 'average',
       'avoid', 'away', 'awesome', 'awsome', 'baby', 'background',
       'badass', 'balance', 'ball', 'bandit', 'bang', 'bank', 'barely',
       'base', 'basic', 'basically', 'batman', 'battle', 'battlefield',
       'beat', 'beautiful', 'begin', 'believe', 'best', 'beta', 'binding',
       'bioshock', 'black', 'blast', 'block', 'blood', 'bloody', 'blow',
       'body', 

In [114]:
# Se convierten los datos obtenidos en un dataframe.
reviews_vector = pd.DataFrame(data=matrix,columns=words)
# Se agrega si se recomienda o no el vídeojuego.
reviews_vector.insert(0,"game_recommendated",review_lemma["recommend"])

reviews_vector

,game_recommendated,ability,able,absolute,absolutely,access,account,achievement,action,actual,...,world,worth,write,wrong,xbox,yeah,year,young,youtube,zombie
0,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,True,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43971,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43972,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43973,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43974,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
